### Рспознавание похожих предметов

In [1]:
import os
from glob import glob
from pymystem3 import Mystem
import sqlite3
import pandas as pd
from pyspark.sql import SQLContext, SparkSession
from pyspark import pandas as ps

/home/al/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [3]:
m = Mystem()
PATH_DD = '/home/al/Projects_My/group_mlada/'
#PATH_DD = '/home/gansiorag/MyParth/ProjectMy/hakaton_ozon_org/'

In [4]:
db_path = PATH_DD + 'dataset/ozon.db'
query = """SELECT name 
FROM sqlite_schema
WHERE type ='table' AND 
    name NOT LIKE 'sqlite_%';"""

conn = sqlite3.connect(db_path)
cur = conn.cursor()
cur.execute(query)
print(cur.fetchall())


[('products',)]


In [5]:
query = """SELECT name FROM PRAGMA_TABLE_INFO('products');"""
cur.execute(query)
for pp in cur.fetchall():
    print(pp[0])

search_nm
item_num
link
short_characteristic
full_desc
img_desc
img_path_desc
characteristic
components
conn_dev
img_path


In [6]:
query = """SELECT search_nm, item_num, link, short_characteristic,full_desc,
img_desc,
img_path_desc,
characteristic,
components,
conn_dev,
img_path 
FROM products;"""
a_pandas_df = pd.read_sql_query(query, conn)

In [7]:
a_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   search_nm             4398 non-null   object
 1   item_num              4398 non-null   object
 2   link                  4398 non-null   object
 3   short_characteristic  4398 non-null   object
 4   full_desc             4398 non-null   object
 5   img_desc              4398 non-null   object
 6   img_path_desc         4398 non-null   object
 7   characteristic        4398 non-null   object
 8   components            4398 non-null   object
 9   conn_dev              4398 non-null   object
 10  img_path              4398 non-null   object
dtypes: object(11)
memory usage: 378.1+ KB


In [8]:
spark = SparkSession.builder.appName('all_dataset').getOrCreate()
print('Spark info :')
spark


24/08/17 13:20:11 WARN Utils: Your hostname, al-Dell resolves to a loopback address: 127.0.1.1; using 192.168.0.106 instead (on interface wlp0s20f3)
24/08/17 13:20:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/17 13:20:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark info :


In [9]:
a_spark_df = ps.from_pandas(a_pandas_df)
df = a_spark_df.to_spark()

/home/al/.local/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [10]:
df.show(5)

24/08/17 13:20:23 WARN TaskSetManager: Stage 0 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


+---------+----------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------+--------------------+
|search_nm|  item_num|                link|short_characteristic|           full_desc|img_desc|       img_path_desc|      characteristic|          components|conn_dev|            img_path|
+---------+----------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------+--------------------+
| смартфон|1409765841|https://www.ozon....|О товаре Перейти ...|Описание Смартфон...|        |dataset/смартфон/...|Характеристики До...|Смартфон, кабель ...|        |dataset/смартфон/...|
| смартфон|1624251355|https://www.ozon....|О товаре Перейти ...|Комплектация Комп...|        |                    |Характеристики До...|Комплект поставки...|        |dataset/смартфон/...|
| смартфон|1337138963|https://www.ozon....|О товаре Перейти 

In [11]:
example_row_full_desc = df.select(['full_desc', 'link']).collect()


24/08/17 13:20:36 WARN TaskSetManager: Stage 1 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


In [12]:
for rr in range(110, 120):
    print(example_row_full_desc[rr]['full_desc'], example_row_full_desc[rr]['link'])

Описание Смартфон Xiaomi Redmi 13C 8/256GB Midnight Black работает на системе Android 13 за счет восьмиядерного процессора. Разрешение экрана составляет 720х1612 пикс. Дисплей оснащен технологией IPS. Оперативная память — 8 Гб, встроенная — 256 Гб. На задней части устройства расположены две основные камеры, позволяющие осуществлять видеосъемку с разрешением 1920х1080 пикс. Устойчивость кадра обеспечивается цифровой стабилизацией. Фронтальная камера — 8 Мп. Для увеличения емкости хранилища данных используется флэш-накопитель microSD с максимальным объемом 512 Гб. Возможно использовать при работе две nano-SIM. Передача данных осуществляется при поддержке стандартов 2G, 3G, 4G LTE, Wi-Fi, NFC, Miracast, Bluetooth 5.0. Доступны функции спутниковой навигации GPS, ГЛОНАСС, встроен FM-тюнер. На задней части панели размещен сканер отпечатка пальца. Корпус устройства сделан из пластика, размещает разъемы 3,5 мм и USB Type-C. В комплект включены блок питания мощностью 10 Вт и провод для зарядки.

In [13]:
type_obj = df.select('search_nm').distinct().collect()
for i in type_obj:
    print(i['search_nm'])

24/08/17 13:21:29 WARN TaskSetManager: Stage 2 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


смартфон
планшет
ноутбук
кроссовки
телевизор
туфли
стул
стол
платье
наушники
чехол для телефона


In [14]:
stop_words = []
with open('/home/al/Projects_My/hakaton_ozon_org/dataset/stop_word_base.txt', 'r') as f:
    for word in f.readlines():
        stop_words.append(word.strip())
len(stop_words)

5499

In [15]:
def func(value):
    for i in range(10):
        value = value.replace(str(i),'')
    value = value.replace('(-)', ' ')
    ddel = ['*', ',', '-', '/', '%', ';', ')', '(', '+', '.', '..', ':', '"',
 'c',
 'h',
 '«',
 '°',
  '–',
 '—',
 '•']
    for i in ddel:
        value = value.replace(i, ' ')
    return value

In [16]:
def get_array_text_with_category(all_text: str, m):
    """ Списка слов из списка текстов

    Args:
        all_text_list (list): Список текстов
        m (_type_): _description_
    """
    sum_arr = []
    words_list = func(all_text.lower()).split()
    for word in words_list:
        # print(word)
        word = word.strip()
        if word:
            lemmas = m.lemmatize(word)
            # print('lemmas[0]', lemmas[0])
            if lemmas[0] not in stop_words:
                print('lemmas[0]', lemmas[0])
                sum_arr.append(lemmas[0])
    return sum_arr

## Получение списков слов по объектам в каждой категориии и создаие наборов train и test.

#### План
1. Распознавание категориальное <br>
2. Распознавание внутрикатегориальное <br>
     <br>
    - по каждой категории свой набор <br>
    - внутри каждой категории наборы по  <br>
        short_characteristic <br>
        full_desc <br>
        characteristic <br>
        components <br>
        с цифрами и без цифр <br> <br>
    <b>распознавание по 9 наборам слов - </b><br>
        общий <br>
        short_characteristic <br>
        full_desc <br>
        characteristic <br>
        components <br>
        short_characteristic <b>с цифрами </b><br>
        full_desc <b>с цифрами </b><br>
        characteristic <b>с цифрами </b><br>
        components <b>с цифрами </b><br>
         <br>

In [17]:
def get_array_words(ddf: dict, m):
    """ Список слов в каждой категории

    Args:
        dict_txt_cat (dict): Словарь категорий
    """
    dict_cat_words = []
    # print(ddf['short_characteristic'])
    dict_cat_words += get_array_text_with_category(ddf['short_characteristic'], m)
    dict_cat_words += get_array_text_with_category(ddf['full_desc'], m)
    dict_cat_words += get_array_text_with_category(ddf['characteristic'], m)
    dict_cat_words += get_array_text_with_category(ddf['components'], m)
    dict_cat_words = set(dict_cat_words)
    return dict_cat_words


In [18]:
len(type_obj)

11

### Получение наборов слов по объектам за вычетом стоп слов

Получение общего словаря.

Получение набора с категориями

In [19]:
array_txt = [] # предложения из лемматизированных слов объектов
m = Mystem()
kk = 0
catY = [] # номера категорий
keys_cat = [] # названия категорий
all_words_list = [] # полный список слов
for sd in type_obj:
    # print(sd )
    kkd = sd['search_nm']
    keys_cat.append(kkd)
    print(kkd)
    list_dir_two = df.filter(df['search_nm'] == kkd).collect()
    # print(len(list_dir_two))
    # print(list_dir_two[10])
    for nf in list_dir_two:
        dd = get_array_words(nf, m)
        all_words_list += dd
        #break
        array_txt.append(' '.join(dd))
        catY.append(kk)
    # print()
    # print(array_txt[kk])
    kk += 1
set_all_words = set(all_words_list)
print(keys_cat)
print(len(array_txt))
print(catY)
print(len(set_all_words))

смартфон


24/08/17 13:22:19 WARN TaskSetManager: Stage 5 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


lemmas[0] макро
lemmas[0] моноблок
lemmas[0] конвертер
lemmas[0] моноблок
lemmas[0] стабилизация
lemmas[0] ростест
lemmas[0] конвертер
lemmas[0] моноблок
lemmas[0] тср
lemmas[0] стабилизация
lemmas[0] портретный
lemmas[0] портретный
lemmas[0] реверсивный
lemmas[0] infinity
lemmas[0] каплевидный
lemmas[0] стабилизация
lemmas[0] моноблок
lemmas[0] моноблок
lemmas[0] телеобъектив
lemmas[0] стабилизация
lemmas[0] ois
lemmas[0] стабилизация
lemmas[0] sju
lemmas[0] ростест
lemmas[0] стабилизация
lemmas[0] моноблок
lemmas[0] ростест
lemmas[0] конвертер
lemmas[0] выводиться
lemmas[0] марафонский
lemmas[0] выносливость
lemmas[0] портретный
lemmas[0] кольцевой
lemmas[0] моноблок
lemmas[0] стабилизация
lemmas[0] vivo
lemmas[0] поликарбонатный
lemmas[0] моноблок
lemmas[0] vivo
lemmas[0] global
lemmas[0] моноблок
lemmas[0] телеобъектив
lemmas[0] стабилизация
lemmas[0] ois
lemmas[0] стабилизация
lemmas[0] sju
lemmas[0] ростест
lemmas[0] выводиться
lemmas[0] марафонский
lemmas[0] выносливость
lemmas[

24/08/17 13:22:25 WARN TaskSetManager: Stage 6 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


lemmas[0] примечательный
lemmas[0] qwerty
lemmas[0] umiio
lemmas[0] umiio
lemmas[0] umiio
lemmas[0] qwerty
lemmas[0] mta
lemmas[0] zunyi
lemmas[0] анонсирование
lemmas[0] xpro
lemmas[0] mtk
lemmas[0] standby
lemmas[0] last
lemmas[0] анонсирование
lemmas[0] spreadtrum
lemmas[0] функционирование
lemmas[0] spreadtrum
lemmas[0] анонсирование
lemmas[0] uwi
lemmas[0] xpro
lemmas[0] uwi
lemmas[0] xpro
lemmas[0] uwi
lemmas[0] xpro
lemmas[0] ·

lemmas[0] ·

lemmas[0] |

lemmas[0] |

lemmas[0] multitou
lemmas[0] |

lemmas[0] ·

lemmas[0] ·

lemmas[0] gmp
lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] uwi
lemmas[0] xpro
lemmas[0] анонсирование
lemmas[0] водозащищенный
lemmas[0] анонсирование
lemmas[0] 🎬
lemmas[0] widevine
lemmas[0] widevine
lemmas[0] тепловыделение
lemmas[0] trans
lemmas[0] анонсирование
lemmas[0] last
lemmas[0] анонсирование
lemmas[0] м

...

lemmas[0] узорчатый
lemmas[0] волнистость
lemmas[0] беззаботный
lemmas[0] олицетворять
lemmas[0] веселие
lemmas[0] щадить
lemmas[0] хлоросодержащий
lemmas[0] муслин
lemmas[0] муслин
lemmas[0] ginger
lemmas[0] декольте
lemmas[0] size
lemmas[0] обхват
lemmas[0] талия
lemmas[0] обхват
lemmas[0] грудь
lemmas[0] обхват
lemmas[0] муслин
lemmas[0] муслин
lemmas[0] ог
lemmas[0] ог
lemmas[0] покрой
lemmas[0] отложной
lemmas[0] ворот
lemmas[0] вверху
lemmas[0] талия
lemmas[0] натали
lemmas[0] ог
lemmas[0] покрой
lemmas[0] ростовка
lemmas[0] индиго
lemmas[0] отбеливание
lemmas[0] запрещать
lemmas[0] эластан
lemmas[0] эластан
lemmas[0] кардиган
lemmas[0] жакет
lemmas[0] треугольный
lemmas[0] zolla
lemmas[0] сарафанов
lemmas[0] комбинезон
lemmas[0] рубашка
lemmas[0] ог
lemmas[0] эластан
lemmas[0] эластан
lemmas[0] ог
lemmas[0] zolla
lemmas[0] покрой
lemmas[0] отбеливать
lemmas[0] барабан
lemmas[0] глажение
lemmas[0] утюг
lemmas[0] глажение
lemmas[0] необратимый
lemmas[0] тетрахлорэтилен
lemmas[0] р

24/08/17 13:23:37 WARN TaskSetManager: Stage 14 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


lemmas[0] aylou
lemmas[0] aylou
lemmas[0] aylou
lemmas[0] сверхнизкий
lemmas[0] моно
lemmas[0] уха
lemmas[0] бежать
lemmas[0] марафон
lemmas[0] усиленно
lemmas[0] потренироваться
lemmas[0] присваиваться
lemmas[0] 💫
lemmas[0] ушной
lemmas[0] раковина
lemmas[0] автоподстройка
lemmas[0] амбушюр
lemmas[0] биопластик
lemmas[0] остановка
lemmas[0] повтор
lemmas[0] штекер
lemmas[0] импеданс
lemmas[0] aylou
lemmas[0] внутриканальный
lemmas[0] freebuds
lemmas[0] цепляться
lemmas[0] амбушюр
lemmas[0] излучатель
lemmas[0] композит
lemmas[0] транспорт
lemmas[0] объявление
lemmas[0] аэропорт
lemmas[0] вокзал
lemmas[0] слышно
lemmas[0] freebuds
lemmas[0] диссонанс
lemmas[0] дважды
lemmas[0] групповой
lemmas[0] life
lemmas[0] внутриканальный
lemmas[0] ушной
lemmas[0] раковина
lemmas[0] multipoint
lemmas[0] амбушюр
lemmas[0] остановка
lemmas[0] freebuds
lemmas[0] всенаправленный
lemmas[0] lda
lemmas[0] амброшур
lemmas[0] интсрукция
lemmas[0] внутриканальный
lemmas[0] режий
lemmas[0] индикация
lemmas[0

24/08/17 13:23:52 WARN TaskSetManager: Stage 15 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


lemmas[0] аквакейс
lemmas[0] герметичность
lemmas[0] опаска
lemmas[0] аквакейс
lemmas[0] гермочехол
lemmas[0] кредитный
lemmas[0] аквакейс
lemmas[0] реалми
lemmas[0] реалми
lemmas[0] реалми
lemmas[0] картхолдер
lemmas[0] карбоновый
lemmas[0] поликарбонат
lemmas[0] бортик
lemmas[0] соприкасаться
lemmas[0] пожелтение
lemmas[0] поликарбонат
lemmas[0] картхолдер
lemmas[0] автомобильный
lemmas[0] asai
lemmas[0] поликарбонат
lemmas[0] банковский
lemmas[0] скидочный
lemmas[0] проездной
lemmas[0] бортик
lemmas[0] kjs
lemmas[0] inf
lemmas[0] trn
lemmas[0] wallet
lemmas[0] over
lemmas[0] ef
lemmas[0] wa
lemmas[0] begru
lemmas[0] бортик
lemmas[0] возвышаться
lemmas[0] olourful
lemmas[0] zes
lemmas[0] хлястик
lemmas[0] редми
lemmas[0] банковский
lemmas[0] визитка
lemmas[0] новаякнижка
lemmas[0] картхолдер
lemmas[0] влияние
lemmas[0] бортик
lemmas[0] обгонять
lemmas[0] дубеть
lemmas[0] плавиться
lemmas[0] противоразводный
lemmas[0] прилипать
lemmas[0] банковский
lemmas[0] банковский
lemmas[0] диско

In [20]:
array_txt

['макро моноблок',
 'стабилизация конвертер моноблок ростест',
 'тср моноблок',
 'моноблок стабилизация infinity портретный реверсивный каплевидный',
 'ростест моноблок sju ois телеобъектив стабилизация',
 'стабилизация конвертер моноблок ростест',
 'марафонский выносливость выводиться моноблок кольцевой стабилизация портретный',
 'vivo поликарбонатный моноблок global',
 'ростест моноблок sju ois телеобъектив стабилизация',
 'марафонский выносливость выводиться моноблок кольцевой стабилизация портретный',
 'стабилизация моноблок ростест',
 'моноблок',
 'виджет светочувствительный \ufeff\ufeff darwin моноблок pdaf ー\n аудиовселенная знакомиться atgpt listen slimming ”\n трехкольцевой ростест разгон стабилизация поэзия исход',
 'стабилизация буклет моноблок лот',
 'жидкостный ростест моноблок лампочка dimensity [\n meteorite стабилизация вибромотор сверхъемкий ー\n осевой',
 'стабилизация моноблок ростест',
 'ростест моноблок неавторизованный вашы стабилизация',
 'победитель голосование м

In [21]:
catY[0]

0

In [21]:
keys_cat[0]

'смартфон'

In [22]:
X_train, X_test, y_train, y_test = train_test_split(array_txt, catY, test_size=0.20, random_state=42)
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)

In [23]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [24]:
# Метод наивный Байес
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [25]:
X_test_new = vectorizer.transform(X_test)
#print(X_test_new.shape)
X_new_tfidf = tfidf_transformer.transform(X_test_new)
predicted = clf.predict(X_new_tfidf)

In [26]:
kk = 0
for doc, category in zip(y_test, predicted):
    # print(doc, category)
    if doc == category: kk +=1
        
print('model ', kk/len(y_test))

model  0.8920454545454546


In [27]:
vectorizer.get_feature_names_out()

array(['aaa', 'aaax', 'aax', ..., 'ячменный', 'ящ', 'ящичек'],
      dtype=object)

In [29]:
print(X_test_new.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Построение распознавания внутри категорий

#### без цифр

In [32]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from time import time

from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [28]:
def get_array_text_with_category2(all_text: str, m):
    """ Списка слов из списка текстов

    Args:
        all_text_list (list): Список текстов
        m (_type_): _description_
    """
    sum_arr = []
    words_list = func(all_text.lower()).split()
    for word in words_list:
        # print(word)
        word = word.strip()
        if word:
            lemmas = m.lemmatize(word)
            # print('lemmas[0]', lemmas[0])
            # if lemmas[0] not in stop_words:
            #     print('lemmas[0]', lemmas[0])
            sum_arr.append(lemmas[0])
    return sum_arr

In [58]:
def get_array_text_with_category_param(df, m, cat: str, param: str):
    list_obj_cat = df.filter(df['search_nm'] == cat).collect()
    array_short_characteristic = []
    num_oby = []
    k = 0
    for obj in list_obj_cat:
        # print(obj[param])
        list_words = get_array_text_with_category2(obj[param], m)
        # print(list_words)
        text_words = ' '.join(list_words)
        array_short_characteristic.append(text_words)
        num_oby.append(k)
        k += 1
    return array_short_characteristic, num_oby, list_obj_cat

In [59]:
cat_desc = ['short_characteristic', 'full_desc', 'characteristic', 'components']
for i in type_obj:
    print(i['search_nm'])

смартфон
планшет
ноутбук
кроссовки
телевизор
туфли
стул
стол
платье
наушники
чехол для телефона


In [60]:
def get_clasters(X_train_sh, y_train_sh):
    vectorizer_sh = CountVectorizer()
    X_train_counts_sh = vectorizer_sh.fit_transform(X_train_sh)
    # X_train_counts_sh    
    kmeans = KMeans(
        n_clusters= 20, # int(len(y_train_sh)),
        max_iter=5,
        # n_init=1,
        # random_state=2,
    ).fit(X_train_counts_sh)
    ones = {}
    for i, y in zip(kmeans.labels_, y_train_sh):
        if i not in ones:
            ones[i] = []
        ones[i].append(y)
    return ones, kmeans, vectorizer_sh

In [64]:
def get_models(cat_i, df, m):
    array_short_characteristic, num_oby, list_obj_cat = get_array_text_with_category_param(df, m, cat_i, 'short_characteristic')
    full_desc, num_oby1, list_obj_cat1 = get_array_text_with_category_param(df, m, cat_i, 'full_desc')
    characteristic, num_oby2, list_obj_cat2 = get_array_text_with_category_param(df, m, cat_i, 'characteristic')
    components, num_oby3, list_obj_cat3 = get_array_text_with_category_param(df, m, cat_i, 'components')
    X_train_array_short_characteristic, X_test_array_short_characteristic, y_train_array_short_characteristic, y_test_array_short_characteristic = train_test_split(array_short_characteristic, num_oby, test_size=0.20, random_state=42)
    X_train_full_desc, X_test_full_desc, y_train_full_desc, y_test_full_desc = train_test_split(full_desc, num_oby, test_size=0.20, random_state=42)
    X_train_characteristic, X_test_characteristic, y_train_characteristic, y_test_characteristic = train_test_split(characteristic, num_oby, test_size=0.20, random_state=42)
    X_train_components, X_test_components, y_train_components, y_test_components = train_test_split(components, num_oby, test_size=0.20, random_state=42)
    answer = {'short_characteristic': {'X_test': X_test_array_short_characteristic,
                                     'y_test': y_test_array_short_characteristic},
            'full_desc': {'X_test': X_test_full_desc,
                          'y_test': y_test_full_desc},
            'characteristic': {'X_test': X_test_characteristic,
                               'y_test': y_test_characteristic},
            # 'components': {'X_test': X_test_components,
            #                'y_test': y_test_components}
            }
    ones, kmeans, vectorizer = get_clasters(X_train_array_short_characteristic, y_train_array_short_characteristic)
    answer['short_characteristic']['ones'] = ones
    answer['short_characteristic']['kmeans'] = kmeans
    answer['short_characteristic']['vectorizer'] = vectorizer
    
    ones, kmeans, vectorizer = get_clasters(X_train_full_desc, y_train_full_desc)
    answer['full_desc']['ones'] = ones
    answer['full_desc']['kmeans'] = kmeans
    answer['full_desc']['vectorizer'] = vectorizer
    
    ones, kmeans, vectorizer = get_clasters(X_train_characteristic, y_train_characteristic)
    answer['characteristic']['ones'] = ones
    answer['characteristic']['kmeans'] = kmeans
    answer['characteristic']['vectorizer'] = vectorizer
    
    # ones, kmeans,  vectorizer = get_clasters(X_train_components, y_train_components)
    # answer['components']['ones'] = ones
    # answer['components']['kmeans'] = kmeans
    # answer['components']['vectorizer'] = vectorizer
    return answer
 
    

In [65]:
array_models = {}
for i in type_obj:
    if i['search_nm'] not in array_models:
        print(i['search_nm'])
        array_models[i['search_nm']] = get_models(i['search_nm'], df, m)


смартфон


24/08/17 14:57:31 WARN TaskSetManager: Stage 120 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:57:32 WARN TaskSetManager: Stage 121 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:57:33 WARN TaskSetManager: Stage 122 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:57:35 WARN TaskSetManager: Stage 123 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


планшет


24/08/17 14:57:36 WARN TaskSetManager: Stage 124 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:57:36 WARN TaskSetManager: Stage 125 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:57:38 WARN TaskSetManager: Stage 126 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:57:40 WARN TaskSetManager: Stage 127 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


ноутбук


...

24/08/17 14:58:29 WARN TaskSetManager: Stage 152 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:58:30 WARN TaskSetManager: Stage 153 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:58:34 WARN TaskSetManager: Stage 154 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:58:38 WARN TaskSetManager: Stage 155 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


наушники


24/08/17 14:58:39 WARN TaskSetManager: Stage 156 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:58:40 WARN TaskSetManager: Stage 157 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:58:45 WARN TaskSetManager: Stage 158 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:58:50 WARN TaskSetManager: Stage 159 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


чехол для телефона


24/08/17 14:58:52 WARN TaskSetManager: Stage 160 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:58:53 WARN TaskSetManager: Stage 161 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:58:56 WARN TaskSetManager: Stage 162 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/17 14:58:58 WARN TaskSetManager: Stage 163 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


In [66]:
import pickle
with open('array_models.pkl', 'wb') as f: 
    pickle.dump(array_models, f)

In [53]:
print(m_smrphone['components'])

{'X_test': ['мобильный телефон коробка инструкция головной зарядный устройство кабель tp гарнитур tp защитный пленка для мобильный телефон защитный чехол штифт для извлечение карта память otg конвертер', 'smart pro вт зарядка кабель скрепка для извлечение слот sim карта инструкция', 'смартфон зарядный устройство документация', 'смартфон кабель usb type адаптер питание инструмент для извлечение сие карта защитный пленка', 'телефон с батарея зарядный устройство usb кабель type устройство для извлечение карта краткий руководство пользователь защитный пленка закреплять на телефон гарантийный талон', 'смартфон зарядный устройство кабель usb a на usb type чехол защитный пленка скрепка для извлечение лоток sim карта руководство пользователь и гарантийный талон', 'смартфон блок питание кабель usb чехол защитный пленка документация', 'смартфон шт кабель шт документация шт', 'мобильный телефон упаковочная коробка инструкция зарядный устройство кабель для передача данный type гарнитур type защитн

In [41]:
def get_classes(kmeans, X_test_sh, y_test_sh, vectorizer_sh):
    # vectorizer_sh = CountVectorizer()
    X_test_sh_counts_sh = vectorizer_sh.transform(X_test_sh)
    # X_test_sh_counts_sh    
    kmeans_p = kmeans.predict(X_test_sh_counts_sh)
    ones = {}
    for i, y in zip(kmeans_p, y_test_sh):
        if i not in ones:
            ones[i] = []
        ones[i].append(y)
    return ones

In [71]:
ones_short_characteristic  = get_classes(kmeans0, X_test_array_short_characteristic, y_test_array_short_characteristic, vectorizer_sh0)
print(ones)

{2: [16, 24, 9, 6, 98, 48, 88, 52, 103], 0: [18, 94, 45, 136, 126, 64, 39, 105, 76, 83, 73, 25, 62, 85, 34, 84, 13, 3, 17, 95, 50, 115, 90, 57, 107, 21, 2, 116, 87, 20, 106], 8: [10, 0, 133, 128, 134], 5: [111, 93, 125, 37], 7: [101], 11: [51, 112, 15, 89, 47, 30, 68, 135, 97, 54, 91, 63, 102], 3: [82, 33, 5, 35, 8, 58, 124, 1, 74], 15: [65, 55, 137], 6: [28, 40, 53, 49, 43, 38, 61, 122, 130, 71, 92], 4: [44, 22, 29], 18: [100, 129], 13: [77, 32], 1: [86, 75], 17: [80, 72, 59, 23], 14: [7, 117, 123, 46, 79, 14], 16: [70], 9: [114], 19: [41], 12: [120, 121, 139], 10: [99]}


In [47]:
ones_full_desc = get_classes(kmeans1, X_test_full_desc, y_test_full_desc, vectorizer_sh1)
print(len(ones_full_desc))


8


In [48]:
ones_characteristic = get_classes(kmeans2, X_test_characteristic, y_test_characteristic, vectorizer_sh2)
print(len(ones_characteristic))


10


In [49]:
ones_components = get_classes(kmeans3, X_test_components, y_test_components, vectorizer_sh3)
print(len(ones_components))

14


In [50]:
def groupping(ones, ones1, ones2, ones3):
    group = {}
    for k in ones:
        #print(k)
        ls = []
        for dd in ones[k]:
            #list_obj_cat[k]['link']
            if list_obj_cat[dd]['link'] not in group:
                group[list_obj_cat[dd]['link']] = []
            ls.append(list_obj_cat[dd]['link'])
        # print(len(group))
        for d2 in ones[k]:
            ff = [dd for dd in ls if dd !=list_obj_cat[d2]['link']]
            # print(len(ff))
            # print(ff)
            # print(list_obj_cat[d2]['link'])
            group[list_obj_cat[d2]['link']].append(set(ff))
            # print(' --> ', len(group[list_obj_cat[d2]['link']]))
    for k in ones1:
        #print(k)
        ls = []
        for dd in ones1[k]:
            #list_obj_cat[k]['link']
            # if list_obj_cat[dd]['link'] not in group:
            #     group[list_obj_cat[dd]['link']] = []
            ls.append(list_obj_cat[dd]['link'])
        # print(len(group))
        for d2 in ones1[k]:
            ff = [dd for dd in ls if dd !=list_obj_cat[d2]['link']]
            # print(len(ff))
            # print(ff)
            # print(list_obj_cat[d2]['link'])
            group[list_obj_cat[d2]['link']].append(set(ff))
            # print(' --> ', len(group[list_obj_cat[d2]['link']]))
    for k in ones2:
        #print(k)
        ls = []
        for dd in ones2[k]:
            #list_obj_cat[k]['link']
            # if list_obj_cat[dd]['link'] not in group:
            #     group[list_obj_cat[dd]['link']] = []
            ls.append(list_obj_cat[dd]['link'])
        # print(len(group))
        for d2 in ones2[k]:
            ff = [dd for dd in ls if dd !=list_obj_cat[d2]['link']]
            # print(len(ff))
            # print(ff)
            # print(list_obj_cat[d2]['link'])
            group[list_obj_cat[d2]['link']].append(set(ff))
            # print(' --> ', len(group[list_obj_cat[d2]['link']]))
    for k in ones3:
        #print(k)
        ls = []
        for dd in ones3[k]:
            #list_obj_cat[k]['link']
            # if list_obj_cat[dd]['link'] not in group:
            #     group[list_obj_cat[dd]['link']] = []
            ls.append(list_obj_cat[dd]['link'])
        # print(len(group))
        for d2 in ones3[k]:
            ff = [dd for dd in ls if dd !=list_obj_cat[d2]['link']]
            # print(len(ff))
            # print(ff)
            # print(list_obj_cat[d2]['link'])
            group[list_obj_cat[d2]['link']].append(set(ff))
            # print(' --> ', len(group[list_obj_cat[d2]['link']]))
    return group

In [51]:
group = groupping(ones, ones1, ones2, ones3)

In [52]:
for k in group:
    print(k)
    print(len(group[k]))
    # for dd in group[k]: 
    #     print(len(dd))

https://www.ozon.ru/product/zentra-smartfon-sp4-rostest-eac-12-64-gb-temno-zelenyy-1601468542/?asb=EWrDgTKLYT70eYhqPMVCBHm0ED6BAUMhZfwna7I%252Bf1g%253D&asb2=0-W_lZAM1tx0as_WoZb0f2hfEOfEbS-1h9fYHeYlNKLpVCTvsBlnx3GQ9MFzFtDdacZhIgcU1d-7GvqvuR9Txw&avtc=1&avte=4&avts=1721078463&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
4
https://www.ozon.ru/product/tecno-smartfon-spark-go-2024-4-64-gb-chernyy-1316485736/?asb=%252FpQw2x26xxkuXIK6cKJCTvOk8bF953QSDeAOMQzoDBg%253D&asb2=F9tqth16BW5d6Ttc9V1Qcd5A4JDAet623jIs4-3pFgiZJZ7vwOFZAVF7hXUptGxo5oP2DyGbdm8Ht2TQIMG_fA&avtc=1&avte=1&avts=1721078453&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
4
https://www.ozon.ru/product/infinix-smartfon-smart-8-4-128-gb-belyy-1429044516/?asb=0L3ES%252FmKe9CZj3IatQE8VxtTEE0dzMEKNmLHBCaZ9PE%253D&asb2=09HHEs905z6MLvBe-TneIl_GwshpmetduAUVuyVfvzPeXtq7QhcfdpvbZdLB7N7xDqVvNAPhCodApCSGCGgrxg&avtc=1&avte=2&avts=1721078393&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
4
https://www.ozon.ru/pro

In [53]:
len(group)

112

In [66]:
def intt(groupl: list):
    sr = []
    for dd in groupl:
        if len(dd) > 0:
            sr.append(dd)
    com = set.intersection(*sr)
    return com


In [139]:
dde = 0
for k in group:
    com = intt(group[k])
    # print(com)
    if dde > 0 :
        print(k)
        ish = analiz_link(k)
        print()
        if len(com) > 0:
            for dd in com:
                srr = analiz_link(dd)
                ind = len(ish.intersection(srr))/len(ish)
                #print(ind)
                if ind > 0.4:
                    print(dd)
                    #print()
            print('='*20)
        else:
            print("No intesepting")
            print('='*20)
            print()
    dde += 1
    if dde > 5: 
        break

https://www.ozon.ru/product/tecno-smartfon-spark-go-2024-4-64-gb-chernyy-1316485736/?asb=%252FpQw2x26xxkuXIK6cKJCTvOk8bF953QSDeAOMQzoDBg%253D&asb2=F9tqth16BW5d6Ttc9V1Qcd5A4JDAet623jIs4-3pFgiZJZ7vwOFZAVF7hXUptGxo5oP2DyGbdm8Ht2TQIMG_fA&avtc=1&avte=1&avts=1721078453&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD

https://www.ozon.ru/product/tecno-smartfon-spark-go-2024-4-64-gb-chernyy-1316485736/?asb=yuyhEuEohZNH5PqFgECDUZQQYDooLfq1EYSbFw7duAo%253D&asb2=wkSV263AfCVvEjhaszW5NERfmtpYtyhKZGha05keoTxklqZ9CMnBIOkWTHAOXZuvvByRir1vxrizD_kcFvIfEQ&avtc=1&avte=1&avts=1721078468&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/infinix-smartfon-smart-8-4-128-gb-belyy-1429044516/?asb=0L3ES%252FmKe9CZj3IatQE8VxtTEE0dzMEKNmLHBCaZ9PE%253D&asb2=09HHEs905z6MLvBe-TneIl_GwshpmetduAUVuyVfvzPeXtq7QhcfdpvbZdLB7N7xDqVvNAPhCodApCSGCGgrxg&avtc=1&avte=2&avts=1721078393&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD

https://www.ozon.ru/product/infinix-smart

In [127]:
dde = 0
for k in group:
    com = intt(group[k])
    # print(com)
    if dde > 5 :
        print(k)
        ish = analiz_link(k)
        print()
        if len(com) > 0:
            for dd in com:
                srr = analiz_link(dd)
                ind = len(ish.intersection(srr))/len(ish)
                print(ind)
                if ind > 0.4:
                    print(dd)
                    print()
            print('='*20)
        else:
            print("No intesepting")
            print('='*20)
            print()
    dde += 1
    if dde > 10: 
        break

https://www.ozon.ru/product/infinix-smartfon-hot-40i-rostest-eac-8-128-gb-chernyy-1413778750/?asb=GhnV3YXsAjtvhCN%252FYg1%252B8fgJ5nZiKnLD7VFpsoSFH2M%253D&asb2=OVUV78yUteBVdzKo-cnSwqwcjXzJyIIEb0zzCz2RaNNbSdC2cuLjuFE8GRF8yXUnBR0ZXRI-_l5Djct2m8neDg&avtc=1&avte=1&avts=1721078448&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD

No intesepting

https://www.ozon.ru/product/tecno-smartfon-spark-go-2024-4-64-gb-chernyy-1316485736/?asb=yuyhEuEohZNH5PqFgECDUZQQYDooLfq1EYSbFw7duAo%253D&asb2=wkSV263AfCVvEjhaszW5NERfmtpYtyhKZGha05keoTxklqZ9CMnBIOkWTHAOXZuvvByRir1vxrizD_kcFvIfEQ&avtc=1&avte=1&avts=1721078468&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD

1.0
https://www.ozon.ru/product/tecno-smartfon-spark-go-2024-4-64-gb-chernyy-1316485736/?asb=%252FpQw2x26xxkuXIK6cKJCTvOk8bF953QSDeAOMQzoDBg%253D&asb2=F9tqth16BW5d6Ttc9V1Qcd5A4JDAet623jIs4-3pFgiZJZ7vwOFZAVF7hXUptGxo5oP2DyGbdm8Ht2TQIMG_fA&avtc=1&avte=1&avts=1721078453&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD

ht

In [72]:
group2 = groupping(ones_short_characteristic, ones_full_desc, ones_characteristic, ones_components)
print(len(group2))

28


In [73]:
group2

{'https://www.ozon.ru/product/zunyi-smartfon-note-30i-rostest-eac-12-256-gb-salatovyy-1599077247/?advert=Al7gxXnUbdwUpvE0tbGjsa0mt_X6N5_0yPSrI1W4kHmrBL6oVacNFbpMUqBNcpwJKUvtjmwLn4646TvpF7RKmxcRePYwGCSwDn3nJ2rRqsDneeDwkV7Jrf0dfDJ6z1Vep_gCk2yJqlzpO2F9jJ0V2wpjD2XzC5GoQbXzTLZuZD_5hFba-Hj5T5nTlk03ICsltaii-5KDw4TTFPUBxRh-hJDy5eSpLkyjgPrL4VPf_m9yFyjm54CCWaFf9noTPvXZpgeb1pyY1Q3ZoyzNtA8RR7-rXS-lGeue1W8nghLzG7Heu_L2BCqhui7XzPPEnsUgPGKabPmOQiB2U-JzH3PpmUAYVxGdUxQPDw8nYfL9yKl4WJtQKurlw-cuguG3xlCjncbfD-ouLw&avtc=1&avte=2&avts=1721078458&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD': [{'https://www.ozon.ru/product/samsung-smartfon-galaxy-s24-ultra-12-256-gb-zheltyy-1398851772/?advert=xF2wHsr4owAVkwsBh0iFhVel473tQedBfO1zVTgnzm7worQk4cD3GHP03PTkYogXFe0XIwzOUz4bln-cws4-MugEFr425WzVkXCRadibh3OadWjvURROk9q6X5oZtQ7dS_FkgaVeAYGem2nTVXEs33jW7Bb0O_TLjXwsXcf3VLbAKb9RPCnYST22G9XrPGCaWANq7Lerdoz7toYuZcl2HdzmoBGccDDed4UOSi-W59SR74xzg8zg1ecD54i2rFPjOzpXASreCMKH40-CGDnEkxeudLwmesmz3SeCTIOpn0vVQZdKKd4k

In [123]:
def analiz_link(ll: str):
    lss = ll.split('product/')[1].split('/?')[0].split('-')[:-1]
    #print(lss)
    return set(lss)

In [122]:
dde2 = 0
for k in group2:
    com = intt(group2[k])
    # print(com)
    if dde2 > 0 :
        print(k)
        ish = analiz_link(k)
        print()
        if len(com) > 0:
            for dd in com:
                srr = analiz_link(dd)
                ind = len(ish.intersection(srr))/len(ish)
                print(ind)
                if ind > 0.4:
                    print(dd)
                    print()
            print('='*20)
        else:
            print("No intesepting")
            print()
    dde2 += 1
    if dde2 > 5: 
        break

https://www.ozon.ru/product/zunyi-smartfon-pove-5-pro-russkaya-globalnaya-versiya-protivoudarnaya-i-vodonepronitsaemaya-1620203481/?advert=d-0ll_aJkfQ2Phjo3afAlHskna-QKufE_54ha396Vn18bo3hSDfB6Hx3cXvfS5vshZQmiVrGgkI6N5z9x109-dvXoQOfsgvVPZYQXsc_zLucZ98CQyAam4h_SOeM52QTInIQBKCwibnjmJJBpPvc9OtlcQUhBg67pRGpR5GxoTAWDNvwkXBhl26Xju0yK9SQ-eWx83gQKwl1LVpPEBC5QNN0pVbOlAxcdKhJYWEdqpmD_NH2vHgvvSUjkQTBd0BMN5cCnjpVhP0ymujENYbWgjh4QmiV9cyckxA7glq51x3CoAjPe_9G0ENTdQcgeMTnGvOsOoFFt_5rkuZxcbk9OuSbPJSWqPa0mjr_MIsPc8yQxdOPecOS3J79-qjrJqFeumlr-3sOqg&avtc=1&avte=2&avts=1721078453&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD

0.45454545454545453
https://www.ozon.ru/product/zunyi-smartfon-pove-5-pro-sverhbolshaya-pamyat-hd-fotografiya-sverhdlinnaya-1617318120/?advert=ve8r3eiowleqAw6lpLkLQqxGop9eep8dQhCTFWtvFhtEI7k1QSIblfGp9Qo0aywGKIJ-VerkBLblTfQNQxEEuNGd_ahRV0ZbzjXMqPO__BIGObEXJiWC91Zxei5hN0l1YJwPONsHLK2C-fdRwRR5vRGH6t-KabYGIrgbraFJ7FMt88gtNFJTbRf4PPrKKnWrfL-dp547E9ehmpRqOcTncmgsfA7Dp35O1Zof977_mw

In [137]:
dde2 = 0
for k in group2:
    com = intt(group2[k])
    # print(com)
    if dde2 > 25 :
        print(k)
        ish = analiz_link(k)
        # print()
        if len(com) > 0:
            for dd in com:
                srr = analiz_link(dd)
                ind = len(ish.intersection(srr))/len(ish)
                #print(ind)
                if ind > 0.5:
                    print(dd)
                    #print()
            print('='*20)
        else:
            print("No intesepting")
            print('='*20)
    dde2 += 1
    if dde2 > 40:
        break

https://www.ozon.ru/product/inoi-smartfon-a62-rostest-eac-2-64-gb-chernyy-1557423108/?asb=7Ka3tTFB7ArZSFX9N2OZuE1wf9Bjc9MSoDRqIVsTEW0%253D&asb2=A5hkR6bHpPqHRYIDYmQb_-KLKtWNMlepInwsGH9J_7m9n8O26eOCCvvWnC1qGkP1yyfMfPqEwcBdeU5McX_tQA&avtc=1&avte=4&avts=1721078458&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
No intesepting
https://www.ozon.ru/product/smartfon-hw-p20pro-b-global-6-128-gb-svetlo-siniy-1548513019/?advert=lVVVxzfThTaIKAnbHfUEZfrLceepjYzYy48AgJ48p3A2ZAqAoGZb48PYwMFUvjpaIVpcMYlb9E_eWvpWxz2DKPOdO-ZaOtOYMZToiEbcXWJx_1Qv1NDyr2ohzeZWMTSct7aZFZKt7jFUBKRw4rpSIEy2LDy1k8h2MuVDrGE9jRzLOoMbR2aRgfxPT16nmb54QP08MO21-80a5bxHhOpT61gGTxYQz6JIKQAEW04T9ohpxCuT8buxtEe8wZkZiUXaaNTNxifSXD11VufThBYgzzIIz7JD5oBbX7gjQyMdleB5c5U8-VnAuimUNbl5yusRsbvg_CR9lvE71K-3-h1mPMhz5lMGlx0tV89qrGFT1iXKwVMRNq48oOjEfoqRzzAxwaL3ZJxOvQ&avtc=1&avte=2&avts=1721078453&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
No intesepting


In [115]:
dde2 = 0
for k in group2:
    com = intt(group2[k])
    # print(com)
    if dde2 > 14:
        print(k)
        ish = analiz_link(k)
        print()
        if len(com) > 0:
            for dd in com:
                srr = analiz_link(dd)
                ind = len(ish.intersection(srr))/len(ish)
                print(ind)
                if ind > 0.5:
                    print(dd)
                    print()
            print('='*20)
        else:
            print("No intesepting")
            print()
    dde2 += 1
    if dde2 > 15: 
        break

https://www.ozon.ru/product/realme-smartfon-note-50-3-64-gb-goluboy-1381734194/?asb=haVWG6Qw8QSxwQkiiykWlxJHcC3eQaZjItVYehdVfRI%253D&asb2=EqDhnDrTjRF1Uu8POU7bhWuGJmjxlhT5D7Tj9F9oUdz84_-9yJGgo9Y5kqEVbZYnoRBGbS-FO-LFXxAOyQlvgg&avtc=1&avte=1&avts=1721078393&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD

0.25
0.25
0.375
https://www.ozon.ru/product/zunyi-smartfon-note-30i-rostest-eac-12-256-gb-salatovyy-1599077247/?advert=Al7gxXnUbdwUpvE0tbGjsa0mt_X6N5_0yPSrI1W4kHmrBL6oVacNFbpMUqBNcpwJKUvtjmwLn4646TvpF7RKmxcRePYwGCSwDn3nJ2rRqsDneeDwkV7Jrf0dfDJ6z1Vep_gCk2yJqlzpO2F9jJ0V2wpjD2XzC5GoQbXzTLZuZD_5hFba-Hj5T5nTlk03ICsltaii-5KDw4TTFPUBxRh-hJDy5eSpLkyjgPrL4VPf_m9yFyjm54CCWaFf9noTPvXZpgeb1pyY1Q3ZoyzNtA8RR7-rXS-lGeue1W8nghLzG7Heu_L2BCqhui7XzPPEnsUgPGKabPmOQiB2U-JzH3PpmUAYVxGdUxQPDw8nYfL9yKl4WJtQKurlw-cuguG3xlCjncbfD-ouLw&avtc=1&avte=2&avts=1721078458&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD

0.375
https://www.ozon.ru/product/xiaomi-smartfon-redmi-note-13-pro-global-8-256-gb